# Visualization


In [ ]:
# !export PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python

import torch
import numpy as np
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from bs_gym.gymbattlesnake import BattlesnakeEnv
from a2c_ppo_acktr.storage import RolloutStorage

import sys
import copy

from policy import SnakePolicyBase, create_policy
from utils import n_opponents, device
from utils import PathHelper, plot_graphs
from performance import check_performance # TODO:


In [ ]:

u = PathHelper()
rollouts = None
policy = None
policies = None

# TODO:
n_envs = 1
n_steps = 600
CPU_THREADS = 6


def setup_rollouts() -> BattlesnakeEnv:
    global rollouts
    tmp_env = BattlesnakeEnv(n_threads=2, n_envs=n_envs)

    rollouts = RolloutStorage(n_steps,
                            n_envs,
                            tmp_env.observation_space.shape,
                            tmp_env.action_space,
                            n_steps)
    tmp_env.close()

    return tmp_env

# TODO: refactor to utils
def setup_policy(tmp_env, model_path=None):
    global device

    policy = create_policy(tmp_env.observation_space.shape, tmp_env.action_space, SnakePolicyBase)
    if model_path is not None:
        policy.load_state_dict(torch.load(model_path))

    policy.to(device)
    policy.eval()

    # TODO: load multiple models, load state dict into policies
    return policy
    


In [ ]:

def get_offset(obs):
    for x in range(23):
        for y in range(23):
            if obs[0][5][x][y] == 1:
                return x, y


def obs_to_frame(obs, width=11, height=11):
    output = np.zeros((width, height, 3), dtype=np.uint8)

    x_offset, y_offset = get_offset(obs)
    
    # layer reference: https://github.com/cbinners/gym-battlesnake/blob/master/gym_battlesnake/src/gamewrapper.cpp#L132
    for x in range(23):
        for y in range(23):
            # Render snake bodies
            if obs[0][1][x][y] == 1:
                output[x-x_offset][y-y_offset] = 255 - 10*(255 - obs[0][2][x][y])

            # layer 4: shared food location
            if obs[0][4][x][y] == 1:
                output[x-x_offset][y-y_offset][0] = 255
                output[x-x_offset][y-y_offset][1] = 255 # yellow
                output[x-x_offset][y-y_offset][2] = 0
            
            # layer 0: snake health
            if obs[0][0][x][y] > 0:
                output[x-x_offset][y-y_offset][0] = 255 # red
                output[x-x_offset][y-y_offset][1] = 0
                output[x-x_offset][y-y_offset][2] = 0
                
                # output[x-x_offset][y-y_offset][3] = obs[0][0][x][y]
                
            # layer 6: agent's head
            if obs[0][6][x][y] == 1:
                output[x-x_offset][y-y_offset][0] = 0
                output[x-x_offset][y-y_offset][1] = 255 # green
                output[x-x_offset][y-y_offset][2] = 0

    return output

def simulate_game(policy, frames=300):
    playground = BattlesnakeEnv(n_threads=CPU_THREADS, n_envs=1, fixed_orientation=True,
                                opponents=policies, device=device)

    # Reset the environment 
    obs = playground.reset()

    obs_list = []

    # Grab a set of frames to render
    with torch.no_grad():
        for _ in tqdm(range(frames)):
            obs_list.append(copy.deepcopy(obs))

            # Get the action our policy should take
            _, action, _, _ = policy.act(torch.tensor(obs, dtype=torch.float32).to(device), None, None)
            print(action)

            # Perform our action and update our observation
            obs,_,_,_ = playground.step(action.cpu().squeeze())

    return obs_list

def visualize_game(obs_list):

    # Keep track of game frames to render
    video = []

    for obs in obs_list:
        # Add the rendered observation to our frame stack
        video.append(obs_to_frame(obs))

    # Render, adapted from here: https://stackoverflow.com/questions/57060422/fast-way-to-display-video-from-arrays-in-jupyter-lab
    video = np.array(video, dtype=np.uint8)
    # fig = plt.figure()
    fig, ax = plt.subplots()

    im = ax.imshow(video[0,:,:,:])

    def init():
        im.set_data(video[0,:,:,:])

    def animate(i):
        for txt in ax.texts:
            txt.remove()
        im.set_data(video[i,:,:,:])

        obs = obs_list[i]
        x_offset, y_offset = get_offset(obs)
        print(f'x_offset: {x_offset}, y_offset: {y_offset}')
        
        layer = 2
        for x in range(23):
            for y in range(23):
                text = ''
                if obs[0][2][x][y] > 0:
                    text = obs[0][layer][x][y]
                ax.text(y-y_offset, x-x_offset, text,
                    ha="center", va="center", color="w")
        return im

    plt.close()

    anim = animation.FuncAnimation(fig,
                                   animate, init_func=init,
                                   frames=video.shape[0],
                                   interval=200 # milliseconds per frame
                                   )
    return anim


In [ ]:
u.set_modelgroup('test10', read_tmp=True)
model_path = None

latest_model_path, cur_iter = u.get_latest_model()
print(latest_model_path)
if latest_model_path is not None:
    model_path = latest_model_path
else:
    sys.exit(1)

print('Loading model from:', model_path)

# print(torch.load(model_path))

tmp_env = setup_rollouts()
policy = setup_policy(tmp_env, model_path=model_path)
if policy is None:
    print('Policy is None')
    sys.exit(1)

policies = [policy for _ in range(n_opponents)]

data = u.load_data(start_iteration=cur_iter)
rewards = data['rewards']
value_losses = data['value_losses']
action_losses = data['action_losses']
dist_entropies = data['dist_entropies']
lengths = data['lengths']

# plot_graphs(rewards, value_losses, action_losses, dist_entropies, lengths)

In [ ]:
# obs_list = simulate_game(policy, frames=1000)
obs_list = simulate_game(policy, frames=10)
anim = visualize_game(obs_list)


In [ ]:
HTML(anim.to_html5_video())

In [ ]:
# TODO: automate finding next best model(s)
models_to_test = [
    # 'models/test10/tmp_iter3000.pt',
    # 'models/test10/tmp_iter2800.pt',
    # 'models/test10/tmp_iter2600.pt',
    # 'models/test10/tmp_iter2400.pt',
    # 'models/test10/tmp_iter2200.pt',
    # 'models/test10/tmp_iter2000.pt',
    # 'models/test10/tmp_iter1800.pt',
    # 'models/test10/tmp_iter1600.pt',
    # 'models/test10/tmp_iter1400.pt',
    # 'models/test10/tmp_iter1200.pt',
    # 'models/test10/tmp_iter1000.pt',
    # 'models/test10/tmp_iter800.pt',
    # 'models/test10/tmp_iter600.pt',
]

def regression_test():
    for next_best_modelpath in models_to_test:
        print(f'Testing {next_best_modelpath}')
        next_best_policy = setup_policy(tmp_env, model_path=next_best_modelpath)
        winrate = check_performance(policy, next_best_policy, device=torch.device(device))
        
        print(f'Winrate: {winrate}')

# regression_test()